# Import important Package

In [9]:
%%capture
import logging
logging.disable(logging.WARNING)

# This fixes colab's default encoding to match huggingface accelerate
import locale
locale.getpreferredencoding = lambda x=False: "UTF-8"
!pip install datasets
!pip install transformers #get transformers
!pip install --upgrade accelerate
!pip install evaluate
!pip install rouge-score #to get the accuracy metric

In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import datasets
from datasets import load_dataset, list_datasets
from evaluate import evaluator
import evaluate
import numpy as np
import torch
import copy

from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_id = 0 if str(device) == 'cuda' else -1

In [3]:
print(device_id)

0


In [10]:
# Download the CNN dataset from huggingface
dataset = load_dataset("cnn_dailymail",'3.0.0',split = 'train') #to get the training

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

Each dataset has these following properties: article, highlights and id. The article is the real article while highlights is the human's notation and id is the unique id for each article

In [5]:
# Download and query the bert-base-cased model
t5_model = pipeline('fill-mask', model='bert-base-cased', device=device_id)
print(bert_model("I went to an [MASK] restaurant and ordered pasta."))